https://gist.github.com/saranshmanu/3e2807409a2838a3e221186ef5528bc7#file-federated_learning-ipynb

Loss function : **hinge_embedding_loss** <br>
Layers : 5 reduce regularly


In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!pip install syft==0.2.8

In [3]:
!pip install torch

In [4]:
import syft as sy 

import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import seaborn as sns
# %matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from syft.frameworks.torch.fl import utils

import random
import time
import json
import copy
import os
import glob

In [5]:
hook = sy.TorchHook(torch)
smart_meter1 = sy.VirtualWorker(hook, id="sm1")
smart_meter2 = sy.VirtualWorker(hook, id="sm2")
compute_nodes = [smart_meter1, smart_meter2]

In [6]:
class Parser:
    def __init__(self):
        self.epochs = 100
        self.lr = 0.001
        self.test_batch_size = 2                                                # number here is [A] and should be equal to [B]
        self.batch_size = 2
        self.log_interval = 10
        self.seed = 1
        self.no_cuda = False
    
args = Parser()
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
device

device(type='cpu')

In [7]:
path='/media/smart_meter/new_data'

In [8]:
data = pd.read_csv(path + "/new_data.csv")

In [9]:
data.head()

,Unnamed: 0,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,...,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season,year
0,1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,...,0,0,0,3,0,0,7.098,10,0,2012
1,2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,...,0,0,0,3,0,0,7.098,10,0,2012
2,3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,...,0,0,0,3,0,0,11.087,10,0,2012
3,4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,...,0,0,0,3,0,0,11.087,10,0,2012
4,5,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,...,0,0,0,3,0,0,13.223,10,0,2012


In [10]:
data = data.drop(['Unnamed: 0','day'], axis = 1)
# print(data[:])

In [11]:
data.head()

,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,...,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season,year
0,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,...,0,0,0,3,0,0,7.098,10,0,2012
1,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,...,0,0,0,3,0,0,7.098,10,0,2012
2,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,...,0,0,0,3,0,0,11.087,10,0,2012
3,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,...,0,0,0,3,0,0,11.087,10,0,2012
4,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,...,0,0,0,3,0,0,13.223,10,0,2012


In [12]:
features = data.drop(['LCLid','energy_sum'], axis = 1)
features = features.to_numpy()  # inputs <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
target = data[['energy_sum']]
target = target.to_numpy()      # output <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

In [13]:
split_frac = 0.8
split_idx= int (len(features)*split_frac)
train_x,test_x = features[:split_idx], features[split_idx:]
train_y,test_y = target[:split_idx], target[split_idx:]

train_y = train_y.ravel()
test_y = test_y.ravel()

print("\t\t\t Feature shapes:")
print("Train set: \t\t{}\n".format(train_x.shape), "Test set: \t\t{}\n".format(test_x.shape))

			 Feature shapes:
Train set: 		(2821320, 32)
 Test set: 		(705330, 32)



In [14]:
train = TensorDataset(torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float())
test = TensorDataset(torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float())
train_loader = DataLoader(train, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=args.test_batch_size, shuffle=True)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64, 52, bias=True)                                          # this number here is [C] and should be equal to [D]
        self.fc2 = nn.Linear(52, 42, bias=True)
        self.fc3 = nn.Linear(42, 32, bias=True)
        self.fc4 = nn.Linear(32, 16, bias=True)
        self.fc5 = nn.Linear(16, 2, bias=True)                                          # this number here is [B] and should be equal to [A]

    def forward(self, x):
        # print(x.shape)  # (8, 32) --> 1D vector of 8*32 = 256
        x = x.view(-1, 64)                                                     # this number here is [D] and should be equal to [C]
        x = F.leaky_relu(self.fc1(x))
        x = F.relu_(self.fc2(x))
        x = F.relu6(self.fc3(x))
        x = F.rrelu(self.fc4(x))
        x = torch.tanh(self.fc5(x)) #self.fc6(x)
        return x

Though data will be available offline for federated learning with the workers but here we are sending the data over to the workers for training with ondevice capability

In [16]:
remote_dataset = (list(), list())
train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    remote_dataset[batch_idx % len(compute_nodes)].append((data, target))

In [17]:
smart_meter1_model = Net()
smart_meter2_model = Net()
smart_meter1_optimizer = optim.SGD(smart_meter1_model.parameters(), lr=args.lr)
smart_meter2_optimizer = optim.SGD(smart_meter2_model.parameters(), lr=args.lr)

In [18]:
models = [smart_meter1_model, smart_meter2_model]
optimizers = [smart_meter1_optimizer, smart_meter2_optimizer]

In [19]:
model = Net()
model

Net(
  (fc1): Linear(in_features=64, out_features=52, bias=True)
  (fc2): Linear(in_features=52, out_features=42, bias=True)
  (fc3): Linear(in_features=42, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=2, bias=True)
)

In [20]:
def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    prediction = model(data)
    loss = F.mse_loss(prediction.view(-1), target)
    # print('prediction', prediction.view(-1).shape)                              # this number here is [E] and should be equal to [F]
    # print('target', target.shape)                                               # this number here is [F] and should be equal to [E]
    loss.backward()
    optimizer.step()
    return model

def train():
    for data_index in range(len(remote_dataset[0])-1):
        for remote_index in range(len(compute_nodes)):
            data, target = remote_dataset[remote_index][data_index]
            models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])
        for model in models:
            model.get()
        return utils.federated_avg({
            "sm1": models[0],
            "sm2": models[1]
        })

In [21]:
def test(federated_model):
    federated_model.eval()
    test_loss = 0
    for data, target in test_loader:
        output = federated_model(data)
        test_loss += F.hinge_embedding_loss(output.view(-1), target, reduction='sum').item()
        predection = output.data.max(1, keepdim=True)[1]
        
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(args.epochs):
    start_time = time.time()
    print(f"Epoch Number {epoch + 1}")
    federated_model = train()
    model = federated_model
    test(federated_model)
    total_time = time.time() - start_time
    print('Communication time over the network', round(total_time, 2), 's\n')

Epoch Number 1
Test set: Average loss: 1.0000
Communication time over the network 149.04 s

Epoch Number 2
Test set: Average loss: 1.0000
Communication time over the network 149.36 s

Epoch Number 3
Test set: Average loss: 1.0000
Communication time over the network 149.18 s

Epoch Number 4
Test set: Average loss: 1.0000
Communication time over the network 166.62 s

Epoch Number 5
Test set: Average loss: 1.0000
Communication time over the network 158.46 s

Epoch Number 6
Test set: Average loss: 1.0000
Communication time over the network 167.28 s

Epoch Number 7
Test set: Average loss: 1.0000
Communication time over the network 149.25 s

Epoch Number 8
Test set: Average loss: 1.0000
Communication time over the network 148.57 s

Epoch Number 9
Test set: Average loss: 1.0000
Communication time over the network 148.15 s

Epoch Number 10
Test set: Average loss: 1.0000
Communication time over the network 148.4 s

Epoch Number 11
Test set: Average loss: 1.0000
Communication time over the netwo